In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np
import requests

In [3]:
requests.get("http://google.com")

<Response [200]>

In [4]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://10.22.50.212:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['10.22.50.212:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [5]:
from src.utils.CacheAdapter import JSONAdapter
from src.utils.DatasetManager import ProjectsDatasetManager

In [6]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [7]:
from random import random
from time import sleep, time
import json

In [8]:
# Validators are used to filter data by quality, 
# for example, I can take only those project, that has long description, readme file and many stars

def projectDataIsSufficient(projectData):
    # filters sufficient data (has description and one(or both) of topics or language)
    return (projectData and projectData["description"] and (len(projectData["topics"]) or projectData["language"]))


USERS_NUMBER_TO_SCAN = 20

def extractScannedUsers(data):
    return list(data.keys())

counter = 0

cacheFileName = "cache__28-03-2025__(sufficient)_{0}.json"

adapter = JSONAdapter(cacheFileName)

ProjectsDatasetManager.usersCollection = usersCollection
ProjectsDatasetManager.projectsCollection = projectsCollection
ProjectsDatasetManager.translatorServers = ["http://3.93.239.17:8000/", "http://54.82.84.209:8000/"]
manager = ProjectsDatasetManager(USERS_NUMBER_TO_SCAN, validate = projectDataIsSufficient, cacheAdapter = adapter)

In [9]:
manager.translateText("你好", 3, True)

'Hello'

In [10]:
with open("/home/trukhinmaksim/src/logs/ignoreUsers.json", encoding = "utf-8") as file:
    manager.ignoreUsers(json.load(file))

print(manager.ignoredUsers)

{'github:betiol': True, 'github:DestinyJun': True, 'github:madf12': True, 'github:hectorqin': True, 'github:yaojunguang': True, 'github:romanofficial': True, 'github:mikedemarais': True, 'github:Redseb': True, 'github:anhquan291': True, 'github:arfanliaqat': True, 'github:zhazihong': True, 'github:xiaomao996688': True, 'github:FelipeMolinari': True, 'github:MohamadKh75': True, 'github:pilotocheg': True, 'github:TPXP': True, 'github:gattleung': True, 'github:872409': True, 'github:yusufbadurohman': True, 'github:EQuimper': True, 'github:neo355': True, 'github:13627491210': True, 'github:MikeChugunov': True, 'github:452MJ': True, 'github:mohamed-sultan': True, 'github:cpojer': True, 'github:Dev4ster': True, 'github:EsunR': True, 'github:guopeng1994': True, 'github:inspiredole': True, 'github:batuhansahan': True, 'github:kolyneh': True, 'github:2017398956': True, 'github:StupidManYI': True, 'github:ddx-510': True, 'github:muhamedzeema': True, 'github:patrickbattisti': True, 'github:BrunoV

In [11]:
startPoint = time()

for i in range(20, 35):
    manager.fromDB()
    manager.preprocess()

    #print(len(manager.data))
    scanned = extractScannedUsers(manager.data)
    manager.ignoreUsers(scanned)

    adapter.collectionName = cacheFileName.format(i)
    #print(adapter.collectionName)
    adapter.save(manager.data)

    counter += len(flatternData(manager.data))

    manager.clearData()

    sleepTime = random() * 15
    print(f"Scanned {USERS_NUMBER_TO_SCAN} users. Sleeping {sleepTime}")
    sleep(sleepTime)

endPoint = time()
print(f"Total scanned: {counter} projects")
print(f"Time spent: {endPoint - startPoint} s")

Scanning user: 0
Scanning user: 1
Scanning user: 2
Scanning user: 3
Scanning user: 4
Scanning user: 5
Scanning user: 6
Scanning user: 7
Scanning user: 8
Scanning user: 9
Scanning user: 10
Scanning user: 11
Scanning user: 12
Scanning user: 13
Scanning user: 14
Scanning user: 15
Scanning user: 16
Scanning user: 17
Scanning user: 18
Scanning user: 19
Scanning user: 20
Scanning user: 21
Scanning user: 22
Scanning user: 23
Scanning user: 24
Scanning user: 25
Scanning user: 26
Scanning user: 27
Scanning user: 28
Scanning user: 29
Scanning user: 30
Scanning user: 31
Scanning user: 32
Scanning user: 33
Scanning user: 34
Scanning user: 35
Scanning user: 36
Scanning user: 37
Scanning user: 38
Scanning user: 39
Scanning user: 40
Scanning user: 41
Scanning user: 42
Scanning user: 43
Scanning user: 44
Scanning user: 45
Scanning user: 46
Scanning user: 47
Scanning user: 48
Scanning user: 49
Scanning user: 50
Scanning user: 51
Scanning user: 52
Scanning user: 53
Scanning user: 54
Scanning user: 55
Sc

In [17]:
#print([*manager.ignoredUsers][220:230])

import json

with open("/home/trukhinmaksim/src/logs/ignoreUsers.json", "w", encoding = "utf-8") as file:
    json.dump([*manager.ignoredUsers], fp = file)

#print()